In [1]:
import qiskit

In [2]:
pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

  Cloning https://github.com/qiskit-community/qiskit-textbook.git to c:\users\anirudh\appdata\local\temp\pip-req-build-zkritd_n
  Created wheel for qiskit-textbook: filename=qiskit_textbook-0.1.0-py3-none-any.whl size=17001 sha256=389171bdfaa30ece13a3bf2912edc864b6f890d03fe17182104b1e22e5234219
  Stored in directory: C:\Users\ANIRUDH\AppData\Local\Temp\pip-ephem-wheel-cache-mhsz4yqc\wheels\80\c4\dd\b7ea1ee2a7247e8997db78bc777bbe22e61ab1e5ed1c4b46e1
Successfully built qiskit-textbook



In [3]:
from qiskit import Aer
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom
import numpy as np

In [8]:
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [5]:
def fidelity(hhl, ref):
    solution_hhl_normed = hhl / np.linalg.norm(hhl)
    solution_ref_normed = ref / np.linalg.norm(ref)
    fidelity = state_fidelity(solution_hhl_normed, solution_ref_normed)
    print("Fidelity:\t\t %f" % fidelity)

In [6]:
matrix = [[1, -1/3], [-1/3, 1]]
vector = [1, 0]

In [10]:
orig_size = len(vector)
matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrix, vector)

# Initialize eigenvalue finding module
eigs = create_eigs(matrix, 3, 50, False)
num_q, num_a = eigs.get_register_sizes()

# Initialize initial state module
init_state = Custom(num_q, state_vector=vector)

# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

In [12]:
result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
print("Solution:\t\t", np.round(result['solution'], 5))

result_ref = NumPyLSsolver(matrix, vector).run()
print("Classical Solution:\t", np.round(result_ref['solution'], 5))

print("Probability:\t\t %f" % result['probability_result'])
fidelity(result['solution'], result_ref['solution'])

Solution:		 [1.13586-0.j 0.40896-0.j]
Classical Solution:	 [1.125 0.375]
Probability:		 0.056291
Fidelity:		 0.999432


In [14]:
print("circuit_width:\t", result['circuit_info']['width'])
print("circuit_depth:\t", result['circuit_info']['depth'])
print("CNOT gates:\t", result['circuit_info']['operations']['cx'])

circuit_width:	 7
circuit_depth:	 101
CNOT gates:	 54


In [17]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np

t = 2  # This is not optimal; As an exercise, set this to the
       # value that will get the best results. See section 8 for solution.

nqubits = 4  # Total number of qubits
nb = 1  # Number of qubits representing the solution
nl = 2  # Number of qubits representing the eigenvalues

theta = 0  # Angle defining |b>

a = 1  # Matrix diagonal
b = -1/3  # Matrix off-diagonal

# Initialise the quantum and classical registers
qr = QuantumRegister(nqubits)

# Create a Quantum Circuit
qc = QuantumCircuit(qr)

qrb = qr[0:nb]
qrl = qr[nb:nb+nl]
qra = qr[nb+nl:nb+nl+1]

# State preparation. 
qc.ry(2*theta, qrb[0])
    
# QPE with e^{iAt}
for qu in qrl:
    qc.h(qu)

qc.p(a*t, qrl[0])
qc.p(a*t*2, qrl[1])

qc.u3(b*t, -np.pi/2, np.pi/2, qrb[0])


# Controlled e^{iAt} on \lambda_{1}:
params=b*t

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

# Controlled e^{2iAt} on \lambda_{2}:
params = b*t*2

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

# Inverse QFT
qc.h(qrl[1])
qc.rz(-np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(-np.pi/4,qrl[0])
qc.h(qrl[0])

# Eigenvalue rotation
t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

qc.cx(qrl[1],qra[0])
qc.ry(t1,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t2,qra[0])
qc.cx(qrl[1],qra[0])
qc.ry(t3,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t4,qra[0])
qc.measure_all()

print("Depth: %i" % qc.depth())
print("CNOTS: %i" % qc.count_ops()['cx'])
qc.draw(fold=100)

Depth: 26
CNOTS: 10


┌───────┐┌───────────────────────┐┌────────┐┌───┐┌──────────────┐┌───┐┌─────────────┐»
 q29_0: ┤ RY(0) ├┤ U3(-0.66667,-π/2,π/2) ├┤ P(π/2) ├┤ X ├┤ RY(-0.66667) ├┤ X ├┤ RY(0.66667) ├»
        └─┬───┬─┘└────────┬──────┬───────┘└────────┘└─┬─┘└──────────────┘└─┬─┘└─────────────┘»
 q29_1: ──┤ H ├───────────┤ P(2) ├────────────────────■────────────────────■─────────────────»
          ├───┤           ├──────┤                                                           »
 q29_2: ──┤ H ├───────────┤ P(4) ├───────────────────────────────────────────────────────────»
          └───┘           └──────┘                                                           »
 q29_3: ─────────────────────────────────────────────────────────────────────────────────────»
                                                                                             »
meas: 4/═════════════════════════════════════════════════════════════════════════════════════»
                                                                                             »
«        ┌─────────┐┌────────┐┌───┐┌─────────────┐┌───┐┌────────────┐┌─────────┐                 »
« q29_0: ┤ P(3π/2) ├┤ P(π/2) ├┤ X ├┤ RY(-1.3333) ├┤ X ├┤ RY(1.3333) ├┤ P(3π/2) ├─────────────────»
«        └─────────┘└────────┘└─┬─┘└─────────────┘└─┬─┘└────────────┘└─────────┘                 »
« q29_1: ───────────────────────┼───────────────────┼──────────────────────────────■─────────────»
«                               │                   │      ┌───┐     ┌──────────┐┌─┴─┐┌─────────┐»
« q29_2: ───────────────────────■───────────────────■──────┤ H ├─────┤ RZ(-π/4) ├┤ X ├┤ RZ(π/4) ├»
«                                                          └───┘     └──────────┘└───┘└─────────┘»
« q29_3: ────────────────────────────────────────────────────────────────────────────────────────»
«                                                                                                »
«meas: 4/════════════════════════════════════════════════════════════════════════════════════════»
«                                                                                                »
«                                                                                       »
« q29_0: ───────────────────────────────────────────────────────────────────────────────»
«             ┌──────────┐     ┌───┐                                                    »
« q29_1: ──■──┤ RZ(-π/4) ├─────┤ H ├────────■────────────────────────────────────────■──»
«        ┌─┴─┐└──────────┘     └───┘        │                                        │  »
« q29_2: ┤ X ├─────■────────────────────────┼────────────────────■───────────────────┼──»
«        └───┘   ┌─┴─┐    ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌─┴─┐»
« q29_3: ────────┤ X ├────┤ RY(-0.69352) ├┤ X ├┤ RY(-0.87728) ├┤ X ├┤ RY(0.35368) ├┤ X ├»
«                └───┘    └──────────────┘└───┘└──────────────┘└───┘└─────────────┘└───┘»
«meas: 4/═══════════════════════════════════════════════════════════════════════════════»
«                                                                                       »
«                       ░ ┌─┐         
« q29_0: ───────────────░─┤M├─────────
«                       ░ └╥┘┌─┐      
« q29_1: ───────────────░──╫─┤M├──────
«                       ░  ║ └╥┘┌─┐   
« q29_2: ───────────────░──╫──╫─┤M├───
«        ┌────────────┐ ░  ║  ║ └╥┘┌─┐
« q29_3: ┤ RY(1.2171) ├─░──╫──╫──╫─┤M├
«        └────────────┘ ░  ║  ║  ║ └╥┘
«meas: 4/══════════════════╩══╩══╩══╩═
«                          0  1  2  3

In [24]:
from qiskit import IBMQ
IBMQ.save_account('85fcd7bcb63cced65aa997fa07d08643b548aa65b022b872166f8ba05cfa35163b8817a8d6df9c723a00a37c9e0ae77a04a0c8b5b3faa22ac67ca79b32e6e7ba')
overwrite=True

configrc.store_credentials:WARNING:2020-12-07 14:40:34,138: Credentials already present. Set overwrite=True to overwrite.


In [25]:
from qiskit import execute, BasicAer, ClassicalRegister, IBMQ
from qiskit.compiler import transpile
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, # Measurement error mitigation functions
                                                 CompleteMeasFitter, 
                                                 MeasurementFilter)

provider = IBMQ.load_account()

backend = provider.get_backend('ibmqx2') # calibrate using real hardware
layout = [2,3,0,4]
chip_qubits = 5

# Transpiled circuit for the real hardware
qc_qa_cx = transpile(qc, backend=backend, initial_layout=layout)

ibmqfactory.load_account:WARNING:2020-12-07 14:40:37,322: Credentials are already in use. The existing account in the session will be replaced.


In [26]:
meas_cals, state_labels = complete_meas_cal(qubit_list=layout, qr=QuantumRegister(chip_qubits))
qcs = meas_cals + [qc_qa_cx]

shots = 10
job = execute(qcs, backend=backend, shots=shots, optimization_level=0)